In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2024-01-23 06:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date_=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [4]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date_=Timestamp('2022-12-27 06:00:00'), to_date_=Timestamp('2023-01-24 06:00:00')
File 2022-12 was already in local storage


File 2023-01 was already in local storage


In [5]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

# add new column pickup_ts with the timestamp in Unix seconds, as datetime features such as pickup_hour 
# are not great type to use for primary_keys, if we use them as primary_keys they may cause potential issues.
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)    
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6 # Unix milliseconds

ts_data

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
100%|██████████| 265/265 [00:02<00:00, 131.09it/s]


,pickup_hour,rides,pickup_location_id,pickup_ts
0,2023-12-26 06:00:00+00:00,0,1,1703570400000
1,2023-12-26 07:00:00+00:00,0,1,1703574000000
2,2023-12-26 08:00:00+00:00,0,1,1703577600000
3,2023-12-26 09:00:00+00:00,0,1,1703581200000
4,2023-12-26 10:00:00+00:00,0,1,1703584800000
...,...,...,...,...
178075,2024-01-23 01:00:00+00:00,3,265,1705971600000
178076,2024-01-23 02:00:00+00:00,5,265,1705975200000
178077,2024-01-23 03:00:00+00:00,0,265,1705978800000
178078,2024-01-23 04:00:00+00:00,0,265,1705982400000


In [6]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/241814
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/178080 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/241814/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f173429d9a0>, None)